# 🔍 EDA: Khám Phá Winner Group - Phân Tích Ngược Dữ Liệu

## Mục tiêu
Sử dụng dữ liệu từ Pancake POS API để **khám phá và hiểu về công ty Winner Group** - một doanh nghiệp bán hàng thời trang chủ yếu là quần áo.

## Phương pháp
- **Phân tích ngược dữ liệu**: Phân tích dữ liệu để suy luận về mô hình kinh doanh, hành vi khách hàng và mô hình vận hành
- **Khám phá dựa trên dữ liệu**: Không có thông tin trước về công ty, chỉ dựa vào dữ liệu thực tế
- **Phân tích toàn diện**: Từ tổng quan đến chi tiết, từ macro đến micro insights

## Cấu trúc EDA
1. **Tổng quan điều hành** - Tổng quan công ty
2. **Phân tích mô hình kinh doanh** - Mô hình kinh doanh
3. **Thông tin khách hàng** - Phân tích khách hàng
4. **Danh mục sản phẩm** - Danh mục sản phẩm
5. **Hiệu quả bán hàng** - Hiệu quả bán hàng
6. **Thông tin vận hành** - Vận hành
7. **Cơ hội tăng trưởng** - Cơ hội tăng trưởng


In [1]:
# ==========================
# 1. Import Thư Viện & Thiết Lập
# ==========================

# Import các thư viện cần thiết cho phân tích dữ liệu
import pandas as pd  # Xử lý dữ liệu dạng bảng
import numpy as np   # Tính toán số học
import matplotlib.pyplot as plt  # Vẽ biểu đồ cơ bản
import seaborn as sns  # Vẽ biểu đồ thống kê đẹp
import plotly.express as px  # Vẽ biểu đồ tương tác
import plotly.graph_objects as go  # Tạo biểu đồ tương tác nâng cao
from plotly.subplots import make_subplots  # Tạo biểu đồ con
import warnings
warnings.filterwarnings('ignore')  # Tắt cảnh báo không cần thiết

# Thiết lập style cho biểu đồ
plt.style.use('seaborn-v0_8')  # Style đẹp cho matplotlib
sns.set_palette("husl")  # Bảng màu đẹp cho seaborn
plt.rcParams['figure.figsize'] = (12, 8)  # Kích thước mặc định
plt.rcParams['font.size'] = 10  # Kích thước font

# Import thư viện kết nối database
import os
from dotenv import load_dotenv  # Đọc biến môi trường từ file .env
from sqlalchemy import create_engine  # Kết nối database

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database từ biến môi trường
DB_USER = os.getenv("DB_USER")  # Tên người dùng database
DB_PASS = os.getenv("DB_PASS")  # Mật khẩu database
DB_HOST = os.getenv("DB_HOST")  # Địa chỉ host database
DB_PORT = os.getenv("DB_PORT")  # Cổng kết nối database
DB_GOLD = os.getenv("DB_GOLD")  # Tên schema Gold

# Tạo engine kết nối tới database Gold
engine_gold = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}"
)

print("✅ Đã import thư viện và kết nối database thành công!")
print(f"📊 Đã kết nối tới schema Gold: {DB_GOLD}")


✅ Đã import thư viện và kết nối database thành công!
📊 Đã kết nối tới schema Gold: winner_gold


## 📈 1. TỔNG QUAN ĐIỀU HÀNH - Winner Group

### Mục tiêu phân tích:
- **Quy mô kinh doanh**: Doanh thu, số đơn hàng, khách hàng
- **Thời gian hoạt động**: Khoảng thời gian dữ liệu có sẵn
- **Tốc độ tăng trưởng**: Xu hướng theo thời gian
- **Hiệu quả kinh doanh**: Các KPI cơ bản


In [2]:
# ==========================
# 1.1. Tải Dữ Liệu Cốt Lõi Từ Tầng Gold
# ==========================

# Tải các bảng fact (bảng sự kiện) - chứa dữ liệu giao dịch
df_orders = pd.read_sql("SELECT * FROM gold_fact_orders", engine_gold)  # Bảng đơn hàng
df_order_items = pd.read_sql("SELECT * FROM gold_fact_order_items", engine_gold)  # Bảng chi tiết sản phẩm trong đơn hàng

# Tải các bảng dimension (bảng chiều) - chứa thông tin mô tả
df_customers = pd.read_sql("SELECT * FROM gold_dim_customers", engine_gold)  # Bảng khách hàng
df_products = pd.read_sql("SELECT * FROM gold_dim_products", engine_gold)  # Bảng sản phẩm
df_pages = pd.read_sql("SELECT * FROM gold_dim_pages", engine_gold)  # Bảng fanpage Facebook
df_shops = pd.read_sql("SELECT * FROM gold_dim_shops", engine_gold)  # Bảng shop
df_date = pd.read_sql("SELECT * FROM gold_dim_date", engine_gold)  # Bảng lịch

print("📊 Đã tải dữ liệu thành công!")
print(f"Đơn hàng: {df_orders.shape[0]:,} bản ghi")
print(f"Chi tiết đơn hàng: {df_order_items.shape[0]:,} bản ghi")
print(f"Khách hàng: {df_customers.shape[0]:,} bản ghi")
print(f"Sản phẩm: {df_products.shape[0]:,} bản ghi")
print(f"Fanpage: {df_pages.shape[0]:,} bản ghi")
print(f"Shop: {df_shops.shape[0]:,} bản ghi")
print(f"Khoảng thời gian: {df_date['date'].min()} đến {df_date['date'].max()}")


📊 Đã tải dữ liệu thành công!
Đơn hàng: 40,236 bản ghi
Chi tiết đơn hàng: 46,611 bản ghi
Khách hàng: 36,090 bản ghi
Sản phẩm: 87 bản ghi
Fanpage: 73 bản ghi
Shop: 1 bản ghi
Khoảng thời gian: 2023-01-01 00:00:00 đến 2026-12-31 00:00:00


In [3]:
# ==========================
# 1.1.1. Khám Phá Cấu Trúc Dữ Liệu - Schema Tầng Gold
# ==========================

print("🔍 KHÁM PHÁ CẤU TRÚC DỮ LIỆU TẦNG GOLD")
print("=" * 60)

# Hàm hiển thị thông tin bảng
def hien_thi_thong_tin_bang(df, ten_bang):
    """
    Hiển thị thông tin chi tiết về cấu trúc của một bảng dữ liệu
    
    Args:
        df: DataFrame cần phân tích
        ten_bang: Tên bảng (tiếng Việt)
    """
    print(f"\n📋 {ten_bang.upper()}")
    print("-" * 40)
    print(f"Kích thước: {df.shape[0]:,} dòng × {df.shape[1]} cột")
    print(f"Các cột: {list(df.columns)}")
    print(f"Kiểu dữ liệu:")
    for col, dtype in df.dtypes.items():
        print(f"  - {col}: {dtype}")
    print(f"Dữ liệu mẫu:")
    print(df.head(3).to_string())
    print("=" * 60)

# Hiển thị thông tin tất cả các bảng
hien_thi_thong_tin_bang(df_orders, "BẢNG ĐƠN HÀNG")
hien_thi_thong_tin_bang(df_order_items, "BẢNG CHI TIẾT ĐƠN HÀNG")
hien_thi_thong_tin_bang(df_customers, "BẢNG KHÁCH HÀNG")
hien_thi_thong_tin_bang(df_products, "BẢNG SẢN PHẨM")
hien_thi_thong_tin_bang(df_pages, "BẢNG FANPAGE")
hien_thi_thong_tin_bang(df_shops, "BẢNG SHOP")
hien_thi_thong_tin_bang(df_date, "BẢNG LỊCH")


🔍 KHÁM PHÁ CẤU TRÚC DỮ LIỆU TẦNG GOLD

📋 BẢNG ĐƠN HÀNG
----------------------------------------
Kích thước: 40,236 dòng × 16 cột
Các cột: ['order_id', 'shop_id', 'page_id', 'customer_id', 'warehouse_id', 'order_date_id', 'status', 'status_name', 'total_quantity', 'gross_revenue', 'net_revenue', 'shipping_fee', 'partner_fee', 'payment_method', 'inserted_at', 'updated_at']
Kiểu dữ liệu:
  - order_id: object
  - shop_id: int64
  - page_id: object
  - customer_id: object
  - warehouse_id: object
  - order_date_id: int64
  - status: int64
  - status_name: object
  - total_quantity: int64
  - gross_revenue: int64
  - net_revenue: int64
  - shipping_fee: int64
  - partner_fee: int64
  - payment_method: object
  - inserted_at: datetime64[ns]
  - updated_at: datetime64[ns]
Dữ liệu mẫu:
  order_id    shop_id          page_id                           customer_id                          warehouse_id  order_date_id  status status_name  total_quantity  gross_revenue  net_revenue  shipping_fee  par

In [4]:
# ==========================
# 1.1.2. Kiểm Tra Chất Lượng Dữ Liệu - Giá Trị Thiếu & Số Lượng Duy Nhất
# ==========================

print("🔍 KIỂM TRA CHẤT LƯỢNG DỮ LIỆU")
print("=" * 60)

# Hàm kiểm tra chất lượng dữ liệu
def kiem_tra_chat_luong_du_lieu(df, ten_bang):
    """
    Kiểm tra chất lượng dữ liệu của một bảng
    
    Args:
        df: DataFrame cần kiểm tra
        ten_bang: Tên bảng (tiếng Việt)
    """
    print(f"\n📊 {ten_bang.upper()} - CHẤT LƯỢNG DỮ LIỆU")
    print("-" * 40)
    
    # Đếm giá trị thiếu (missing values)
    gia_tri_thieu = df.isnull().sum()
    ty_le_thieu = (gia_tri_thieu / len(df)) * 100
    
    # Đếm số lượng giá trị duy nhất
    so_luong_duy_nhat = df.nunique()
    
    # Tạo báo cáo chất lượng
    bao_cao_chat_luong = pd.DataFrame({
        'Cột': df.columns,
        'Số_Giá_Trị_Thiếu': gia_tri_thieu.values,
        'Tỷ_Lệ_Thiếu_%': ty_le_thieu.values,
        'Số_Lượng_Duy_Nhất': so_luong_duy_nhat.values,
        'Kiểu_Dữ_Liệu': df.dtypes.values
    })
    
    print(bao_cao_chat_luong.to_string(index=False))
    print("=" * 60)

# Kiểm tra tất cả các bảng
kiem_tra_chat_luong_du_lieu(df_orders, "BẢNG ĐƠN HÀNG")
kiem_tra_chat_luong_du_lieu(df_order_items, "BẢNG CHI TIẾT ĐƠN HÀNG")
kiem_tra_chat_luong_du_lieu(df_customers, "BẢNG KHÁCH HÀNG")
kiem_tra_chat_luong_du_lieu(df_products, "BẢNG SẢN PHẨM")
kiem_tra_chat_luong_du_lieu(df_pages, "BẢNG FANPAGE")
kiem_tra_chat_luong_du_lieu(df_shops, "BẢNG SHOP")
kiem_tra_chat_luong_du_lieu(df_date, "BẢNG LỊCH")


🔍 KIỂM TRA CHẤT LƯỢNG DỮ LIỆU

📊 BẢNG ĐƠN HÀNG - CHẤT LƯỢNG DỮ LIỆU
----------------------------------------
           Cột  Số_Giá_Trị_Thiếu  Tỷ_Lệ_Thiếu_%  Số_Lượng_Duy_Nhất   Kiểu_Dữ_Liệu
      order_id                 0       0.000000              40236         object
       shop_id                 0       0.000000                  1          int64
       page_id                10       0.024853                 73         object
   customer_id                 0       0.000000              36080         object
  warehouse_id                 0       0.000000                 24         object
 order_date_id                 0       0.000000                503          int64
        status                 0       0.000000                  7          int64
   status_name                 0       0.000000                  7         object
total_quantity                 0       0.000000                 19          int64
 gross_revenue                 0       0.000000                442     

In [5]:
# ==========================
# 1.2. Tổng Quan Điều Hành - Các Chỉ Số Chính
# ==========================

# Chuyển đổi cột ngày tháng sang định dạng datetime
df_orders['inserted_at'] = pd.to_datetime(df_orders['inserted_at'])
df_orders['updated_at'] = pd.to_datetime(df_orders['updated_at'])

# Tính toán các chỉ số chính
tong_doanh_thu = df_orders['gross_revenue'].sum()  # Tổng doanh thu
tong_don_hang = len(df_orders)  # Tổng số đơn hàng
tong_khach_hang = len(df_customers)  # Tổng số khách hàng
tong_san_pham = len(df_products[df_products['is_dummy'] == 0])  # Loại trừ sản phẩm giả
tong_fanpage = len(df_pages)  # Tổng số fanpage

# Khoảng thời gian hoạt động
khoang_thoi_gian = df_orders['inserted_at'].max() - df_orders['inserted_at'].min()
so_ngay_hoat_dong = khoang_thoi_gian.days

# Các chỉ số trung bình
gia_tri_don_hang_tb = tong_doanh_thu / tong_don_hang  # Giá trị đơn hàng trung bình
don_hang_moi_khach_tb = tong_don_hang / tong_khach_hang  # Số đơn hàng trung bình mỗi khách
doanh_thu_moi_khach_tb = tong_doanh_thu / tong_khach_hang  # Doanh thu trung bình mỗi khách

print("🏢 WINNER GROUP - TỔNG QUAN ĐIỀU HÀNH")
print("=" * 50)
print(f"💰 Tổng Doanh Thu: {tong_doanh_thu:,.0f} VND ({tong_doanh_thu/1e9:.2f}T VND)")
print(f"📦 Tổng Đơn Hàng: {tong_don_hang:,}")
print(f"👥 Tổng Khách Hàng: {tong_khach_hang:,}")
print(f"🛍️ Tổng Sản Phẩm: {tong_san_pham:,}")
print(f"📱 Tổng Fanpage: {tong_fanpage:,}")
print(f"📅 Thời Gian Hoạt Động: {so_ngay_hoat_dong} ngày")
print("=" * 50)
print(f"💵 Giá Trị Đơn Hàng TB: {gia_tri_don_hang_tb:,.0f} VND")
print(f"🔄 Đơn Hàng/Khách TB: {don_hang_moi_khach_tb:.2f}")
print(f"💎 Doanh Thu/Khách TB: {doanh_thu_moi_khach_tb:,.0f} VND")
print("=" * 50)


🏢 WINNER GROUP - TỔNG QUAN ĐIỀU HÀNH
💰 Tổng Doanh Thu: 16,545,782,490 VND (16.55T VND)
📦 Tổng Đơn Hàng: 40,236
👥 Tổng Khách Hàng: 36,090
🛍️ Tổng Sản Phẩm: 37
📱 Tổng Fanpage: 73
📅 Thời Gian Hoạt Động: 1324 ngày
💵 Giá Trị Đơn Hàng TB: 411,218 VND
🔄 Đơn Hàng/Khách TB: 1.11
💎 Doanh Thu/Khách TB: 458,459 VND


In [6]:
# ==========================
# 1.3. Phân Tích Xu Hướng Doanh Thu
# ==========================

# Tạo phân tích dựa trên ngày tháng
df_orders['ngay_don_hang'] = df_orders['inserted_at'].dt.date  # Ngày đơn hàng
df_orders['thang_don_hang'] = df_orders['inserted_at'].dt.to_period('M')  # Tháng đơn hàng

# Xu hướng doanh thu theo ngày
doanh_thu_theo_ngay = df_orders.groupby('ngay_don_hang')['gross_revenue'].sum().reset_index()
doanh_thu_theo_ngay['ngay_don_hang'] = pd.to_datetime(doanh_thu_theo_ngay['ngay_don_hang'])

# Xu hướng doanh thu theo tháng
doanh_thu_theo_thang = df_orders.groupby('thang_don_hang')['gross_revenue'].sum().reset_index()
doanh_thu_theo_thang['thang_chuoi'] = doanh_thu_theo_thang['thang_don_hang'].astype(str)

# Tạo biểu đồ tương tác
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Xu Hướng Doanh Thu Theo Ngày', 'Xu Hướng Doanh Thu Theo Tháng'),
    vertical_spacing=0.1
)

# Biểu đồ xu hướng theo ngày
fig.add_trace(
    go.Scatter(
        x=doanh_thu_theo_ngay['ngay_don_hang'],
        y=doanh_thu_theo_ngay['gross_revenue'],
        mode='lines+markers',
        name='Doanh Thu Theo Ngày',
        line=dict(color='#1f77b4', width=2),
        marker=dict(size=4)
    ),
    row=1, col=1
)

# Biểu đồ xu hướng theo tháng
fig.add_trace(
    go.Bar(
        x=doanh_thu_theo_thang['thang_chuoi'],
        y=doanh_thu_theo_thang['gross_revenue'],
        name='Doanh Thu Theo Tháng',
        marker_color='#ff7f0e'
    ),
    row=2, col=1
)

fig.update_layout(
    title='📈 Xu Hướng Doanh Thu Winner Group',
    height=600,
    showlegend=False
)

fig.update_xaxes(title_text="Ngày", row=1, col=1)
fig.update_yaxes(title_text="Doanh Thu (VND)", row=1, col=1)
fig.update_xaxes(title_text="Tháng", row=2, col=1)
fig.update_yaxes(title_text="Doanh Thu (VND)", row=2, col=1)

# Hiển thị biểu đồ
fig

# In ra các thông tin insights
print("📊 THÔNG TIN XU HƯỚNG DOANH THU:")
print(f"📅 Khoảng Thời Gian: {doanh_thu_theo_ngay['ngay_don_hang'].min().strftime('%Y-%m-%d')} đến {doanh_thu_theo_ngay['ngay_don_hang'].max().strftime('%Y-%m-%d')}")
print(f"💰 Ngày Tốt Nhất: {doanh_thu_theo_ngay.loc[doanh_thu_theo_ngay['gross_revenue'].idxmax(), 'ngay_don_hang'].strftime('%Y-%m-%d')} ({doanh_thu_theo_ngay['gross_revenue'].max():,.0f} VND)")
print(f"📈 Tháng Tốt Nhất: {doanh_thu_theo_thang.loc[doanh_thu_theo_thang['gross_revenue'].idxmax(), 'thang_chuoi']} ({doanh_thu_theo_thang['gross_revenue'].max():,.0f} VND)")
print(f"📊 Doanh Thu Trung Bình Ngày: {doanh_thu_theo_ngay['gross_revenue'].mean():,.0f} VND")
print(f"📊 Doanh Thu Trung Bình Tháng: {doanh_thu_theo_thang['gross_revenue'].mean():,.0f} VND")


📊 THÔNG TIN XU HƯỚNG DOANH THU:
📅 Khoảng Thời Gian: 2021-12-30 đến 2025-08-16
💰 Ngày Tốt Nhất: 2024-12-12 (166,911,020 VND)
📈 Tháng Tốt Nhất: 2024-12 (3,092,223,120 VND)
📊 Doanh Thu Trung Bình Ngày: 32,894,200 VND
📊 Doanh Thu Trung Bình Tháng: 590,920,803 VND


## 🏢 2. PHÂN TÍCH MÔ HÌNH KINH DOANH

### Mục tiêu phân tích:
- **Kênh bán hàng**: Phân tích hiệu quả các fanpage Facebook
- **Phương thức thanh toán**: Tỷ lệ sử dụng các hình thức thanh toán
- **Địa lý**: Phân bố khách hàng theo khu vực
- **Mô hình hoạt động**: Cách thức vận hành kinh doanh


## 👥 3. THÔNG TIN KHÁCH HÀNG - Phân Tích Khách Hàng

### Mục tiêu phân tích:
- **Phân khúc khách hàng**: Phân khúc khách hàng theo RFM và trạng thái VIP
- **Nhân khẩu học**: Phân tích giới tính, địa lý
- **Phân tích hành vi**: Hành vi mua hàng, tần suất, giá trị
- **Giá trị khách hàng trọn đời**: Giá trị khách hàng theo thời gian


In [7]:
# ==========================
# 3.1. Phân Tích Nhân Khẩu Học Khách Hàng
# ==========================

# Chuyển đổi cột ngày tháng cho khách hàng
df_customers['last_order_at'] = pd.to_datetime(df_customers['last_order_at'], errors='coerce')

# Phân bố giới tính
phan_bo_gioi_tinh = df_customers['gender'].value_counts()
ty_le_gioi_tinh = df_customers['gender'].value_counts(normalize=True) * 100

# Khách hàng VIP vs Thường
phan_bo_vip = df_customers['is_vip'].value_counts()
ty_le_vip = df_customers['is_vip'].value_counts(normalize=True) * 100

# Tạo biểu đồ trực quan
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Phân Bố Giới Tính', 'Khách Hàng VIP vs Thường', 
                   'Phân Bố Giá Trị Khách Hàng', 'Phân Bố Thời Gian Gần Nhất'),
    specs=[[{"type": "pie"}, {"type": "pie"}],
           [{"type": "histogram"}, {"type": "histogram"}]]
)

# Biểu đồ tròn giới tính
fig.add_trace(
    go.Pie(
        labels=phan_bo_gioi_tinh.index,
        values=phan_bo_gioi_tinh.values,
        name='Giới Tính',
        hole=0.3
    ),
    row=1, col=1
)

# Biểu đồ tròn VIP
fig.add_trace(
    go.Pie(
        labels=['Thường', 'VIP'],
        values=phan_bo_vip.values,
        name='Trạng Thái VIP',
        hole=0.3
    ),
    row=1, col=2
)

# Phân bố giá trị khách hàng
fig.add_trace(
    go.Histogram(
        x=df_customers['purchased_amount'],
        nbinsx=50,
        name='Giá Trị Khách Hàng',
        marker_color='#FF6B6B'
    ),
    row=2, col=1
)

# Phân bố thời gian gần nhất
fig.add_trace(
    go.Histogram(
        x=df_customers['recency_days'],
        nbinsx=50,
        name='Số Ngày Gần Nhất',
        marker_color='#4ECDC4'
    ),
    row=2, col=2
)

fig.update_layout(
    title='👥 Phân Tích Nhân Khẩu Học & Hành Vi Khách Hàng',
    height=800,
    showlegend=False
)

# Hiển thị biểu đồ
fig

# In ra các thông tin insights
print("👥 THÔNG TIN NHÂN KHẨU HỌC KHÁCH HÀNG:")
print(f"👫 Phân Bố Giới Tính:")
for gioi_tinh, so_luong in phan_bo_gioi_tinh.items():
    ty_le = ty_le_gioi_tinh[gioi_tinh]
    print(f"   - {gioi_tinh}: {so_luong:,} khách hàng ({ty_le:.1f}%)")

print(f"\n💎 Trạng Thái VIP:")
for trang_thai, so_luong in phan_bo_vip.items():
    ty_le = ty_le_vip[trang_thai]
    ten_trang_thai = "VIP" if trang_thai == 1 else "Thường"
    print(f"   - {ten_trang_thai}: {so_luong:,} khách hàng ({ty_le:.1f}%)")

print(f"\n📊 Thống Kê Giá Trị Khách Hàng:")
print(f"   - Trung Bình: {df_customers['purchased_amount'].mean():,.0f} VND")
print(f"   - Trung Vị: {df_customers['purchased_amount'].median():,.0f} VND")
print(f"   - Tối Đa: {df_customers['purchased_amount'].max():,.0f} VND")

print(f"\n📅 Thống Kê Thời Gian Gần Nhất:")
print(f"   - Trung Bình: {df_customers['recency_days'].mean():.1f} ngày")
print(f"   - Trung Vị: {df_customers['recency_days'].median():.1f} ngày")
print(f"   - Khách hàng hoạt động (≤30 ngày): {(df_customers['recency_days'] <= 30).sum():,}")
print(f"   - Khách hàng có nguy cơ (31-90 ngày): {((df_customers['recency_days'] > 30) & (df_customers['recency_days'] <= 90)).sum():,}")
print(f"   - Khách hàng không hoạt động (>90 ngày): {(df_customers['recency_days'] > 90).sum():,}")


👥 THÔNG TIN NHÂN KHẨU HỌC KHÁCH HÀNG:
👫 Phân Bố Giới Tính:
   - Nữ: 19,486 khách hàng (54.0%)
   - Nam: 11,239 khách hàng (31.1%)
   - Khác: 5,365 khách hàng (14.9%)

💎 Trạng Thái VIP:
   - Thường: 28,767 khách hàng (79.7%)
   - VIP: 7,323 khách hàng (20.3%)

📊 Thống Kê Giá Trị Khách Hàng:
   - Trung Bình: 222,193 VND
   - Trung Vị: 0 VND
   - Tối Đa: 6,199,000 VND

📅 Thống Kê Thời Gian Gần Nhất:
   - Trung Bình: 328.2 ngày
   - Trung Vị: 432.0 ngày
   - Khách hàng hoạt động (≤30 ngày): 9,414
   - Khách hàng có nguy cơ (31-90 ngày): 0
   - Khách hàng không hoạt động (>90 ngày): 26,676


In [8]:
# ==========================
# 3.2. Phân Tích RFM - Phân Khúc Khách Hàng
# ==========================

# Tạo phân khúc RFM
def tao_phan_khuc_rfm(df):
    """
    Tạo phân khúc RFM (Recency, Frequency, Monetary) cho khách hàng
    
    Args:
        df: DataFrame chứa thông tin khách hàng
        
    Returns:
        DataFrame với các cột phân khúc RFM
    """
    # Phân khúc Recency (Thời gian gần nhất)
    df['phan_khuc_recency'] = pd.cut(df['recency_days'], 
                                    bins=[-1, 30, 90, 180, float('inf')], 
                                    labels=['Hoạt Động', 'Có Nguy Cơ', 'Không Hoạt Động', 'Mất'])
    
    # Phân khúc Frequency (Tần suất)
    df['phan_khuc_frequency'] = pd.cut(df['frequency'], 
                                      bins=[0, 1, 3, 10, float('inf')], 
                                      labels=['Thấp', 'Trung Bình', 'Cao', 'Rất Cao'])
    
    # Phân khúc Monetary (Giá trị tiền tệ)
    df['phan_khuc_monetary'] = pd.cut(df['monetary'], 
                                     bins=[0, 100000, 500000, 1000000, float('inf')], 
                                     labels=['Thấp', 'Trung Bình', 'Cao', 'Rất Cao'])
    
    return df

# Áp dụng phân khúc RFM
df_khach_hang_rfm = tao_phan_khuc_rfm(df_customers.copy())

# Tạo biểu đồ RFM
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Phân Khúc Recency', 'Phân Khúc Frequency', 
                   'Phân Khúc Monetary', 'Ma Trận RFM'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Phân khúc Recency
dem_recency = df_khach_hang_rfm['phan_khuc_recency'].value_counts()
fig.add_trace(
    go.Bar(
        x=dem_recency.index,
        y=dem_recency.values,
        name='Recency',
        marker_color='#FF6B6B'
    ),
    row=1, col=1
)

# Phân khúc Frequency
dem_frequency = df_khach_hang_rfm['phan_khuc_frequency'].value_counts()
fig.add_trace(
    go.Bar(
        x=dem_frequency.index,
        y=dem_frequency.values,
        name='Frequency',
        marker_color='#4ECDC4'
    ),
    row=1, col=2
)

# Phân khúc Monetary
dem_monetary = df_khach_hang_rfm['phan_khuc_monetary'].value_counts()
fig.add_trace(
    go.Bar(
        x=dem_monetary.index,
        y=dem_monetary.values,
        name='Monetary',
        marker_color='#45B7D1'
    ),
    row=2, col=1
)

# Biểu đồ scatter RFM
fig.add_trace(
    go.Scatter(
        x=df_khach_hang_rfm['frequency'],
        y=df_khach_hang_rfm['monetary'],
        mode='markers',
        marker=dict(
            size=8,
            color=df_khach_hang_rfm['recency_days'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Số Ngày Gần Nhất")
        ),
        name='Ma Trận RFM'
    ),
    row=2, col=2
)

fig.update_layout(
    title='📊 Phân Tích RFM - Phân Khúc Khách Hàng',
    height=800,
    showlegend=False
)

# Hiển thị biểu đồ
fig

# In ra thông tin RFM
print("📊 THÔNG TIN PHÂN TÍCH RFM:")
print(f"🔄 Phân Khúc Recency:")
for phan_khuc, so_luong in dem_recency.items():
    ty_le = (so_luong / len(df_khach_hang_rfm)) * 100
    print(f"   - {phan_khuc}: {so_luong:,} khách hàng ({ty_le:.1f}%)")

print(f"\n📈 Phân Khúc Frequency:")
for phan_khuc, so_luong in dem_frequency.items():
    ty_le = (so_luong / len(df_khach_hang_rfm)) * 100
    print(f"   - {phan_khuc}: {so_luong:,} khách hàng ({ty_le:.1f}%)")

print(f"\n💰 Phân Khúc Monetary:")
for phan_khuc, so_luong in dem_monetary.items():
    ty_le = (so_luong / len(df_khach_hang_rfm)) * 100
    print(f"   - {phan_khuc}: {so_luong:,} khách hàng ({ty_le:.1f}%)")

# Top khách hàng theo RFM
print(f"\n🏆 TOP KHÁCH HÀNG THEO RFM:")
top_khach_hang = df_khach_hang_rfm.nlargest(10, 'monetary')[['customer_id', 'frequency', 'monetary', 'recency_days', 'is_vip']]
print(top_khach_hang.to_string(index=False))


📊 THÔNG TIN PHÂN TÍCH RFM:
🔄 Phân Khúc Recency:
   - Mất: 26,626 khách hàng (73.8%)
   - Không Hoạt Động: 50 khách hàng (0.1%)
   - Có Nguy Cơ: 0 khách hàng (0.0%)
   - Hoạt Động: 0 khách hàng (0.0%)

📈 Phân Khúc Frequency:
   - Thấp: 32,594 khách hàng (90.3%)
   - Trung Bình: 3,390 khách hàng (9.4%)
   - Cao: 103 khách hàng (0.3%)
   - Rất Cao: 1 khách hàng (0.0%)

💰 Phân Khúc Monetary:
   - Trung Bình: 10,002 khách hàng (27.7%)
   - Cao: 6,382 khách hàng (17.7%)
   - Rất Cao: 431 khách hàng (1.2%)
   - Thấp: 72 khách hàng (0.2%)

🏆 TOP KHÁCH HÀNG THEO RFM:
                         customer_id  frequency  monetary  recency_days  is_vip
49240427-99a6-430d-9ec4-c058b8688b89          2   6199000         265.0       1
b2609ca1-0827-4e3e-b864-d9cb22415f32          5   4940000         422.0       1
0ca048c6-7e2c-4563-be78-75b5131c5740          3   4230000         270.0       1
69529dd7-a4b6-41d4-9d17-11b55c27e42d          2   3740000         542.0       1
6afb7ed5-0e2a-4b5d-9267-4d1f17f02c0

In [9]:
# ==========================
# 2.1. Phân Tích Hiệu Quả Fanpage Facebook
# ==========================

# Kết hợp dữ liệu đơn hàng với dữ liệu fanpage
df_don_hang_fanpage = df_orders.merge(df_pages, on='page_id', how='left')

# Tính toán hiệu quả fanpage
hieu_qua_fanpage = df_don_hang_fanpage.groupby(['page_id', 'page_name']).agg({
    'gross_revenue': 'sum',  # Tổng doanh thu
    'order_id': 'count',     # Số đơn hàng
    'total_quantity': 'sum'  # Tổng số lượng
}).reset_index()

# Đặt tên cột
hieu_qua_fanpage.columns = ['page_id', 'page_name', 'tong_doanh_thu', 'tong_don_hang', 'tong_so_luong']
hieu_qua_fanpage['gia_tri_don_hang_tb'] = hieu_qua_fanpage['tong_doanh_thu'] / hieu_qua_fanpage['tong_don_hang']
hieu_qua_fanpage = hieu_qua_fanpage.sort_values('tong_doanh_thu', ascending=False)

# Top 10 fanpage hiệu quả nhất
top_fanpage = hieu_qua_fanpage.head(10)

# Tạo biểu đồ trực quan
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Top 10 Fanpage Theo Doanh Thu', 'Top 10 Fanpage Theo Đơn Hàng', 
                   'Phân Bố Doanh Thu', 'Phân Bố Đơn Hàng'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Biểu đồ doanh thu
fig.add_trace(
    go.Bar(
        x=top_fanpage['page_name'],
        y=top_fanpage['tong_doanh_thu'],
        name='Doanh Thu',
        marker_color='#2E8B57'
    ),
    row=1, col=1
)

# Biểu đồ đơn hàng
fig.add_trace(
    go.Bar(
        x=top_fanpage['page_name'],
        y=top_fanpage['tong_don_hang'],
        name='Đơn Hàng',
        marker_color='#4682B4'
    ),
    row=1, col=2
)

# Phân bố doanh thu
fig.add_trace(
    go.Histogram(
        x=hieu_qua_fanpage['tong_doanh_thu'],
        nbinsx=20,
        name='Phân Bố Doanh Thu',
        marker_color='#FF6347'
    ),
    row=2, col=1
)

# Phân bố đơn hàng
fig.add_trace(
    go.Histogram(
        x=hieu_qua_fanpage['tong_don_hang'],
        nbinsx=20,
        name='Phân Bố Đơn Hàng',
        marker_color='#9370DB'
    ),
    row=2, col=2
)

fig.update_layout(
    title='📱 Phân Tích Hiệu Quả Fanpage Facebook',
    height=800,
    showlegend=False
)

fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)

# Hiển thị biểu đồ
fig

# In ra thông tin insights
print("📱 THÔNG TIN FANPAGE FACEBOOK:")
print(f"🏆 Fanpage Hiệu Quả Nhất: {top_fanpage.iloc[0]['page_name']}")
print(f"💰 Doanh Thu Fanpage Tốt Nhất: {top_fanpage.iloc[0]['tong_doanh_thu']:,.0f} VND")
print(f"📦 Đơn Hàng Fanpage Tốt Nhất: {top_fanpage.iloc[0]['tong_don_hang']:,}")
print(f"📊 Tổng Fanpage Hoạt Động: {len(hieu_qua_fanpage)}")
print(f"📈 Doanh Thu Trung Bình Mỗi Fanpage: {hieu_qua_fanpage['tong_doanh_thu'].mean():,.0f} VND")
print(f"📊 Đơn Hàng Trung Bình Mỗi Fanpage: {hieu_qua_fanpage['tong_don_hang'].mean():.1f}")


📱 THÔNG TIN FANPAGE FACEBOOK:
🏆 Fanpage Hiệu Quả Nhất: Gạo Linen
💰 Doanh Thu Fanpage Tốt Nhất: 2,243,873,900 VND
📦 Đơn Hàng Fanpage Tốt Nhất: 5,637
📊 Tổng Fanpage Hoạt Động: 73
📈 Doanh Thu Trung Bình Mỗi Fanpage: 226,584,541 VND
📊 Đơn Hàng Trung Bình Mỗi Fanpage: 551.0
